## 2 Layer MLP
- With hidden layer of size 64 and dropout with 0.5 probability
- Softmax at last classification layer 
- Learning rate of 0.001
- Over 50-100 epochs

In [1]:
import numpy as np
import tensorflow as tf
import argparse
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
import math

encoder = load_model("../Models/ae_encoder_mixed.keras")
X = np.load("../Data/LPS/sal-lps-150-1500(labeled)_data.npy")
Y = np.load("../Data/LPS/sal-lps-150-1500(labeled)_labels.npy")


2025-09-10 23:04:27.703466: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-10 23:04:37.381392: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-09-10 23:04:37.446348: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 23:04:37.742639: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 23:04:37.939736: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 1349980000 exceeds 10% of free system memory.
2025-09-10 23:04:49.406196: W external/local_x

In [2]:
encoder.trainable = False
encoder.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 5000)           │   337,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │     5,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       500,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 343,001,500 (1.28 GB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 343,001,500 (1.28 GB)

In [3]:
class MLPClassifier(tf.keras.Model):
    def __init__(self, encoder, num_classes=2, hidden_size=64, dropout_rate=0.5):
        super().__init__()
        self.encoder = encoder
        self.encoder.trainable = False  # Freeze encoder
        self.hidden = layers.Dense(hidden_size, activation='relu')
        self.dropout = layers.Dropout(dropout_rate)
        self.classifier = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.encoder(inputs)
        x = self.hidden(x)
        x = self.dropout(x)
        return self.classifier(x)

In [4]:
# Split into train (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)

# Split temp into validation (10%) and test (20%)
# Since temp is 30%, validation should be 1/3 of temp (10%), test 2/3 of temp (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)

In [5]:
print(f'Training data shape: {X_train.shape}')
print(f'Val data shape: {X_val.shape}')
print(f'Test data shape: {X_test.shape}')


Training data shape: (2291, 67499)
Val data shape: (327, 67499)
Test data shape: (656, 67499)


In [6]:
model = MLPClassifier(encoder)

early_stopping = EarlyStopping(
    monitor='accuracy',
    patience=3,
    restore_best_weights=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
            )

print("Training started....")
model.fit(
    X_train, y_train,
    epochs=50, batch_size=32,    # Use 50-100 epochs as needed
    validation_data=(X_val, y_val),  # Use validation labels here
    verbose=1,
    callbacks=[early_stopping]
)
print("Training completed!")


Training started....
Epoch 1/50


2025-09-10 23:14:13.227119: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 618560836 exceeds 10% of free system memory.


72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 138ms/step - accuracy: 0.5591 - loss: 0.6805 - val_accuracy: 0.7034 - val_loss: 0.6546
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 125ms/step - accuracy: 0.6805 - loss: 0.6424 - val_accuracy: 0.7462 - val_loss: 0.6098
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 129ms/step - accuracy: 0.7246 - loss: 0.5986 - val_accuracy: 0.7645 - val_loss: 0.5695
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 126ms/step - accuracy: 0.7481 - loss: 0.5596 - val_accuracy: 0.7676 - val_loss: 0.5361
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 135ms/step - accuracy: 0.7761 - loss: 0.5241 - val_accuracy: 0.8165 - val_loss: 0.5051
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 130ms/step - accuracy: 0.7866 - loss: 0.4949 - val_accuracy: 0.8165 - val_loss: 0.4818
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step - accuracy: 0.8075 - loss: 0.4686 - val_accuracy: 0.8165 - val_loss: 0.4615
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.8210 - loss: 0.4484 - val_accuracy: 0.8196 - 

In [ ]:
some_metric = model.evaluate(X_test,y_test)
print(some_metric)

21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.8613 - loss: 0.3252
[0.32515403628349304, 0.8612805008888245]
